In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, ensemble
from IPython.display import display

### Load Data

In [2]:
# create date parser
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')

# create data type converter
dtype_map = dict(Station = 'str')

# read data into PANDAS DataFrames with date parsing
test = pd.read_csv('input/test.csv', parse_dates=['Date'], date_parser=dateparse)
train = pd.read_csv('input/train.csv', parse_dates=['Date'], date_parser=dateparse, dtype= dtype_map)
weather = pd.read_csv('input/weather.csv', parse_dates=['Date'], date_parser=dateparse, dtype= dtype_map)
sample_sub = pd.read_csv('input/sampleSubmission.csv')

In [3]:
print('Train')
display(train.info())

print('Test')
display(test.info())

Train
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
Date                      10506 non-null datetime64[ns]
Address                   10506 non-null object
Species                   10506 non-null object
Block                     10506 non-null int64
Street                    10506 non-null object
Trap                      10506 non-null object
AddressNumberAndStreet    10506 non-null object
Latitude                  10506 non-null float64
Longitude                 10506 non-null float64
AddressAccuracy           10506 non-null int64
NumMosquitos              10506 non-null int64
WnvPresent                10506 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(4), object(5)
memory usage: 779.8+ KB


None

Test
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116293 entries, 0 to 116292
Data columns (total 11 columns):
Id                        116293 non-null int64
Date                      116293 non-null datetime64[ns]
Address                   116293 non-null object
Species                   116293 non-null object
Block                     116293 non-null int64
Street                    116293 non-null object
Trap                      116293 non-null object
AddressNumberAndStreet    116293 non-null object
Latitude                  116293 non-null float64
Longitude                 116293 non-null float64
AddressAccuracy           116293 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(3), object(5)
memory usage: 7.5+ MB


None

In [4]:
print('Weather')
display(weather.info())

Weather
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
Station        2944 non-null object
Date           2944 non-null datetime64[ns]
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2944 non-null object
Depart         2944 non-null object
DewPoint       2944 non-null int64
WetBulb        2944 non-null object
Heat           2944 non-null object
Cool           2944 non-null object
Sunrise        2944 non-null object
Sunset         2944 non-null object
CodeSum        2944 non-null object
Depth          2944 non-null object
Water1         2944 non-null object
SnowFall       2944 non-null object
PrecipTotal    2944 non-null object
StnPressure    2944 non-null object
SeaLevel       2944 non-null object
ResultSpeed    2944 non-null float64
ResultDir      2944 non-null int64
AvgSpeed       2944 non-null object
dtypes: datetime64[ns](1), float64(1), int64(4), object(16)
memory usage: 322.0+ KB


None

### Select Columns

In [5]:
# weather
weather_exclude = ['Dewpoint', 'WetBulb', 'CodeSum', 'Depth', 'Water1', 'SnowFall', 'StnPressure',
                 'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed','DewPoint']
weather_cols = [col for col in weather.columns if col not in weather_exclude]
weather = weather[weather_cols]


# train
train_exclude = ['Address', 'AddressNumberAndStreet', 'AddressAccuracy', 'NumMosquitos']
train_cols = [col for col in train.columns if col not in train_exclude]
train = train[train_cols]

# test
test_exclude = ['Address', 'AddressNumberAndStreet', 'AddressAccuracy', 'Id']
test_cols = [col for col in test.columns if col not in test_exclude]
test = test[test_cols]

In [6]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 11 columns):
Station        2944 non-null object
Date           2944 non-null datetime64[ns]
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2944 non-null object
Depart         2944 non-null object
Heat           2944 non-null object
Cool           2944 non-null object
Sunrise        2944 non-null object
Sunset         2944 non-null object
PrecipTotal    2944 non-null object
dtypes: datetime64[ns](1), int64(2), object(8)
memory usage: 161.0+ KB


In [7]:
print('Weather')
display(weather.head())

print('Train')
display(train.head())

Weather


,Station,Date,Tmax,Tmin,Tavg,Depart,Heat,Cool,Sunrise,Sunset,PrecipTotal
0,1,2007-05-01,83,50,67,14,0,2,0448,1849,0.00
1,2,2007-05-01,84,52,68,M,0,3,-,-,0.00
2,1,2007-05-02,59,42,51,-3,14,0,0447,1850,0.00
3,2,2007-05-02,60,43,52,M,13,0,-,-,0.00
4,1,2007-05-03,66,46,56,2,9,0,0446,1851,0.00


Train


,Date,Species,Block,Street,Trap,Latitude,Longitude,WnvPresent
0,2007-05-29,CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,41.954690,-87.800991,0
1,2007-05-29,CULEX RESTUANS,41,N OAK PARK AVE,T002,41.954690,-87.800991,0
2,2007-05-29,CULEX RESTUANS,62,N MANDELL AVE,T007,41.994991,-87.769279,0
3,2007-05-29,CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,41.974089,-87.824812,0
4,2007-05-29,CULEX RESTUANS,79,W FOSTER AVE,T015,41.974089,-87.824812,0


In [8]:
# what species have been detected (note that according to the CDC each
# of these species can carry WNV)
set(train.Species)

{'CULEX ERRATICUS',
 'CULEX PIPIENS',
 'CULEX PIPIENS/RESTUANS',
 'CULEX RESTUANS',
 'CULEX SALINARIUS',
 'CULEX TARSALIS',
 'CULEX TERRITANS'}

### Examine and Handle missing Data

**What is 'T' and 'M'?**
- From http://www.nws.noaa.gov/om/csd/info/NOWdata/FAQ.php

> "M" stands for "Missing". Data for an element will be missing if the primary sensor for that weather element is inoperable (e.g., has an outage) or malfunctioning (e.g., producing errant data) AND any collocated backup sensor is also inoperable or malfunctioning. "T" stand for "Trace". This is a small amount of precipitation that will wet a raingage but is less than the 0.01 inch measuring limit.

The Precipitation Total column is the only column in the retained data that can contain this value. The value has leading whitespace so we should strip prior to counting.

In [9]:
# strip whitespace
weather.PrecipTotal = weather.PrecipTotal.str.strip()

In [10]:
miss_weather = ['M', '-']
trace_weather = ['T']

In [11]:
cols_not_date = [col for col in weather.columns if col != 'Date']

In [12]:
weather[cols_not_date].apply(pd.value_counts, axis=1)[miss_weather + trace_weather].fillna(0).sum()

M    1507.0
-    2944.0
T     318.0
dtype: float64

In [13]:
# Both stations
check = weather[cols_not_date].apply(pd.value_counts, axis=0).fillna(0)
check.loc[['M', '-', 'T']]

,Station,Tmax,Tmin,Tavg,Depart,Heat,Cool,Sunrise,Sunset,PrecipTotal
M,0.0,0.0,0.0,11.0,1472.0,11.0,11.0,0.0,0.0,2.0
-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1472.0,1472.0,0.0
T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,318.0


In [14]:
# Station 1
check_stat1 = weather[cols_not_date][weather.Station == '1'].apply(pd.value_counts, axis=0).fillna(0)
check_stat1.loc[['M', '-', 'T']]

,Station,Tmax,Tmin,Tavg,Depart,Heat,Cool,Sunrise,Sunset,PrecipTotal
M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,163.0


In [15]:
# Station 2
check_stat2 = weather[cols_not_date][weather.Station == '2'].apply(pd.value_counts, axis=0).fillna(0)
check_stat2.loc[['M', '-', 'T']]

,Station,Tmax,Tmin,Tavg,Depart,Heat,Cool,Sunrise,Sunset,PrecipTotal
M,0.0,0.0,0.0,11.0,1472.0,11.0,11.0,0.0,0.0,2.0
-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1472.0,1472.0,0.0
T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,155.0


In [16]:
# Both stations
check.loc[['M', '-', 'T']]/(len(weather)) * 100

,Station,Tmax,Tmin,Tavg,Depart,Heat,Cool,Sunrise,Sunset,PrecipTotal
M,0.0,0.0,0.0,0.373641,50.0,0.373641,0.373641,0.0,0.0,0.067935
-,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,50.0,50.0,0.000000
T,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,10.801630


In [17]:
# Station 1
check_stat1.loc[['M', '-', 'T']]/(len(weather)) * 100

,Station,Tmax,Tmin,Tavg,Depart,Heat,Cool,Sunrise,Sunset,PrecipTotal
M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.536685


In [18]:
# Station 2()
check_stat2.loc[['M', '-', 'T']]/(len(weather)) * 100

,Station,Tmax,Tmin,Tavg,Depart,Heat,Cool,Sunrise,Sunset,PrecipTotal
M,0.0,0.0,0.0,0.373641,50.0,0.373641,0.373641,0.0,0.0,0.067935
-,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,50.0,50.0,0.000000
T,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,5.264946


In [19]:
weather = weather.replace('M', np.NaN)
weather = weather.replace('-', np.NaN)
weather = weather.replace('T', 0.005) # very small amounts of rain can impact mosquito hatches

In [20]:
# convert datatpypes

to_numeric = ['Tmax','Tmin','Tavg', 'Depart', 'Heat', 'Cool', 'PrecipTotal']

for col in to_numeric:
    weather[col]= pd.to_numeric(weather[col])

In [21]:
weather.Sunrise = weather.Sunrise.fillna(method = 'ffill')
weather.Sunset = weather.Sunset.fillna(method = 'ffill')

In [81]:
for index, val in enumerate(weather.Sunset):
    try:
        pd.to_datetime(val, format = '%H%M').time()
    except:
        print(index, val, val[2:])

(20, '1860', '60')
(21, '1860', '60')
(276, '1760', '60')
(277, '1760', '60')
(348, '1660', '60')
(349, '1660', '60')
(388, '1860', '60')
(389, '1860', '60')
(644, '1760', '60')
(645, '1760', '60')
(716, '1660', '60')
(717, '1660', '60')
(756, '1860', '60')
(757, '1860', '60')
(1012, '1760', '60')
(1013, '1760', '60')
(1084, '1660', '60')
(1085, '1660', '60')
(1124, '1860', '60')
(1125, '1860', '60')
(1380, '1760', '60')
(1381, '1760', '60')
(1452, '1660', '60')
(1453, '1660', '60')
(1492, '1860', '60')
(1493, '1860', '60')
(1748, '1760', '60')
(1749, '1760', '60')
(1820, '1660', '60')
(1821, '1660', '60')
(1860, '1860', '60')
(1861, '1860', '60')
(2116, '1760', '60')
(2117, '1760', '60')
(2188, '1660', '60')
(2189, '1660', '60')
(2228, '1860', '60')
(2229, '1860', '60')
(2484, '1760', '60')
(2485, '1760', '60')
(2556, '1660', '60')
(2557, '1660', '60')
(2596, '1860', '60')
(2597, '1860', '60')
(2852, '1760', '60')
(2853, '1760', '60')
(2924, '1660', '60')
(2925, '1660', '60')


In [80]:
for val in weather.Sunset:
    if val[2:] == '60':
        val = val.replace(val[2:], '59')
        print val

1859
1859
1759
1759
1659
1659
1859
1859
1759
1759
1659
1659
1859
1859
1759
1759
1659
1659
1859
1859
1759
1759
1659
1659
1859
1859
1759
1759
1659
1659
1859
1859
1759
1759
1659
1659
1859
1859
1759
1759
1659
1659
1859
1859
1759
1759
1659
1659


In [22]:
time_func = lambda x: pd.to_datetime(x, format = '%H%M').time()

In [ ]:
weather.Sunrise = weather.Sunrise.apply(time_func)

In [ ]:
weather.Sunset = weather.Sunset.apply(time_func)

In [ ]:
weather.info()

In [ ]:
weather.describe()

### Create Month and Day columns

In [ ]:
month_func = lambda x: x.month
day_func= lambda x: x.day
day_of_year_func = lambda x: x.dayofyear

train['month'] = train.Date.apply(month_func)
train['day'] = train.Date.apply(day_func)
train['day_of_year'] = train.Date.apply(day_of_year_func)
test['month'] = test.Date.apply(month_func)
test['day'] = test.Date.apply(day_func)
test['day_of_year'] = test.Date.apply(day_of_year_func)

In [ ]:
train.day_of_year.value_counts()

### Create integer latitude and longitude columns

### Merge Data

In [ ]:
train = train.merge(weather, on='Date')
test = test.merge(weather, on='Date')
train = train.drop(['Date'], axis = 1)
test = test.drop(['Date'], axis = 1)

### Handle Weather Stations 1

In [ ]:
# split the data into two dataframes by station

train_station_1= train[train.Station == 1]
train_station_2= train[train.Station == 2]

In [ ]:
# export to JSON for external use
train_station_1.to_json('train_station_1.json')
train_station_2.to_json('train_station_2.json')
train.to_json('train.json')

# epxort to csv for external use
train_station_1.to_csv('train_station_1.csv')
train_station_2.to_csv('train_station_2.csv')
train.to_csv('train.csv')


In [ ]:
ls